In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from src.L_functions import get_vol, get_neg_sharpe, rast
from src.Models import Multi_CBO_model, gen_Psi,CBO_model
from src.utils import Lmin, avg_x, avg_L, load_config, next_run_index, save_run_config
import random
import os
import sys
from tqdm import trange  # pip install tqdm

In [2]:
def has_converged_all_coords_equal(X: np.ndarray, tol: float = 1e-6) -> bool:
    """
    X: (N, D) 파티클 행렬
    tol: 좌표별 허용 오차. max-min <= tol 이면 같은 값으로 간주.
    """
    # np.ptp = max - min, axis=0 → 각 좌표(l)별 범위
    return np.all(np.ptp(X, axis=0) <= tol)

In [3]:
RESULTS_ROOT = Path("./results")
SIMNAME_SUB  = "250901RAST_Sigma"


# ---- 메인 ----
# np.random.seed(1)
base_cfg   = load_config("configs/configRAST.yaml")
config = dict(base_cfg)
# x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
# np.save( "x0.npy", x0)
# x0 = np.load("x0.npy")

lam1 = 0 
sigma = 0 
for sigma in [0, 1, 2,3,4,]:
    print(sigma)
    run_idx  = next_run_index(RESULTS_ROOT, SIMNAME_SUB)
    run_name = f"{SIMNAME_SUB}_{run_idx}"
    run_root = RESULTS_ROOT / run_name           # 상위 simname 폴더 (여기 아래에 sim0000 ... 생성)
    run_root.mkdir(parents=True, exist_ok=True)

    # 원본 yaml 직접 덮지 말고 메모리에서 수정
    config = dict(base_cfg)
    config["sigma"]    = sigma
    config["simname"] = run_name
    config["lam1"] = lam1
    save_run_config(run_root, config)

    # 문제 정의

    L = rast



    # 공용 초기값 (원하면 실험별 동일 초기값)
    # x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
    # np.save(run_root / "x0.npy", x0)

    nsim    = int(config['sim'])
    maxiter = int(config.get('maxiter', 1000))

    # 진행률: 바깥 루프(시뮬레이션 개수)
    for simnum in range(nsim):
        # 개별 sim 폴더 생성 ...
        sim_dir  = run_root / f"sim{simnum:04d}"
        figs_dir = sim_dir / "figs"
        figs_dir.mkdir(parents=True, exist_ok=True)
    
        model = CBO_model(L, config)
        model.path = str(sim_dir.resolve()) + "/"
        model.make_path()
    
        # x = np.load( "x0.npy")
        x = 2 * np.random.rand(config['nump'], config['D']) + 2
        E = np.zeros(maxiter, dtype=float)
    
        for it in range(maxiter):
    
            if config.get('model', 1) == 1:
                x = model.step(x, L)
            else:
                raise NotImplementedError("model!=1 분기 사용 시 다른 모델 지정 필요")
    
            model.trace_func(x, lambda z: z, "coord")
            model.trace_func(x, lambda x: model.best_loss(x, L), "best_L")
            # --- 진행률 (print 버전) ---
            # if (it + 1) % 50 == 0 or it == maxiter - 1:
            #     pct = (simnum * maxiter + it + 1) / (nsim * maxiter) * 100
            #     print(f"\rProgress: {pct:.1f}%", end="")
            if has_converged_all_coords_equal(x,1e-4):

                break
        # 시뮬레이션 종료 후 개행
        print(it)
        model.save_func("coord", simnum=simnum)
        model.save_func("best_L", simnum=simnum)
        # eps = 1e-12
        # plt.figure()
        # plt.plot(np.arange(maxiter), E + eps, label='E_n')
        # plt.yscale('log')
        # plt.title(f'sim{simnum:04d}')
        # plt.xlabel('iter')
        # plt.ylabel('energy')
        # plt.legend()
        # plt.savefig(figs_dir / "energy.png", dpi=150, bbox_inches="tight")
        # plt.close()


0
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
1
179
152
218
145
151
186
147
146
154
162
162
173
164
177
144
154
159
157
171
167
163
171
170
178
173
178
145
202
154
169
165
169
143
171
165
140
197
149
141
195
175
161
152
202
171
206
157
157
143
144
2
335
262
332
326
257
360
324
292
353
380
408
341
401
433
521
429
294
368
340
396
353
393
397
283
446
405
322
400
292
289
455
343
406
345
361
463
337
393
470
358
501
367
414
325
466
311
412
380
327
516
3
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
4
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999


In [21]:
from src.trace_stats import summarize_scalar_trace, summarize_coord_trace, save_stats_json

run_root = "results/250901RAST_Lam1_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_stats = summarize_scalar_trace(run_root, funcname="best_L")
print("best_L mean[-10:]:", bestL_stats["mean"][-10:])
save_stats_json(bestL_stats, f"{run_root}/stats/best_L_stats.json")

# 2) coord (N,D) 통계
coord_stats = summarize_coord_trace(run_root, funcname="coord",
                                    reduce_over="particles_and_sims")
print("coord mean shape:", coord_stats["mean"].shape)  # (T, D)
save_stats_json(coord_stats, f"{run_root}/stats/coord_stats.json")

best_L mean[-10:]: [12.17377215 12.17388534 12.17398767 12.17408014 12.17416366 12.17423908
 12.17430715 12.17436858 12.17442399 12.17447397]
coord mean shape: (94, 15)


In [22]:
run_root = "results/250901RAST_Lam1_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

12.174473968012128
1.5990289252786989


In [23]:
run_root = "results/250901RAST_Lam1_1"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

9.240300946279678
0.8537060928607634


In [24]:
run_root = "results/250901RAST_Lam1_2"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

8.327667440253162
0.7539113595991962


In [25]:
run_root = "results/250901RAST_Lam1_3"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

7.782543190251637
0.8713263780114124


In [26]:
run_root = "results/250901RAST_Lam1_4"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

7.800282748675412
1.1612651555301525


In [27]:
run_root = "results/250901RAST_Lam1_5"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

7.607883430544479
0.8923308138230056


In [28]:
run_root = "results/250901RAST_Lam1_6"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

6.941875378188697
0.8821265647433528


In [29]:
run_root = "results/250901RAST_Lam1_7"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

7.006073719873931
0.9905109178607489


In [30]:
run_root = "results/250901RAST_Lam1_8"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

7.076906665678557
1.1233456027339097


In [31]:
run_root = "results/250901RAST_Lam1_9"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

6.8728911101813965
1.1803314435313466


In [32]:
run_root = "results/250901RAST_Lam1_10"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

6.746794708142394
1.5050057209216536


In [33]:
run_root = "results/250901RAST_Sigma_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

12.28158903756386
1.296179267317333


In [34]:
run_root = "results/250901RAST_Sigma_1"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

8.587260613161936
0.7420258236198349


In [35]:
run_root = "results/250901RAST_Sigma_2"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

3.1311904666378445
0.8519939602848629


In [36]:
run_root = "results/250901RAST_Sigma_3"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

0.02629110649728862
0.03742017580572429


In [37]:
vrun_root = "results/250901RAST_Sigma_4"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])

0.02629110649728862
0.03742017580572429
